<a href="https://colab.research.google.com/github/hayat415/repo/blob/main/Flower_assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import zipfile
import numpy as np
import random
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from keras.preprocessing import image
import tensorflow as tf
from google.colab import files
from keras import models
from keras import layers
from keras import regularizers
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount()

KeyboardInterrupt: ignored

In [ ]:
local_zip = ""
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall("/flower_new")
zip_ref.close()

In [ ]:
  fname = os.listdir(source)
  images= None
  
  for i,value in enumerate(fname):
    source_path = os.path.join(source, fname[i])
    if os.path.getsize(source_path)==0:
      print("file has zero size")
    else:
      img=image.load_img(source_path, target_size=(28, 28)) # Load image
      x=image.img_to_array(img)                               # Image to array RGB
      x=tf.image.rgb_to_grayscale(x)                          # gray scale conversion
      x=np.expand_dims(x, axis=0)
      if images is None:
        images = x
      else:
        images = np.vstack((images, x))
  print(images.shape)
  images=images.reshape(images.shape[0], 28*28*1)
  print(images.shape)
  label_in = np.ones((len(fname), 1))
  label_in.fill(label)
  print(label_in.shape)
  images_new=np.hstack((images,label_in))
  print(images_new.shape)
  return images_new

In [ ]:
daisy_source_dir = "/flower_new/flowers/daisy"

dandelion_source_dir = "/flower_new/flowers/dandelion"

rose_source_dir = "/flower_new/flowers/rose"

sunflower_source_dir = "/flower_new/flowers/sunflower"

tulip_source_dir = "/flower_new/flowers/tulip"

In [ ]:
daisy = convert_image(daisy_source_dir, 0)
dandelion = convert_image(dandelion_source_dir, 1)
rose = convert_image(rose_source_dir, 2)
sunflower = convert_image(sunflower_source_dir, 3)
tulip = convert_image(tulip_source_dir, 4)

In [ ]:
data_array = np.vstack((daisy, dandelion, rose, sunflower, tulip))
data_array.shape

In [ ]:
np.random.shuffle(data_array) # Shuffling the data
train_data_full= data_array[:2540,:]
test_data_full = data_array[2540:, :]
train_data = train_data_full[:, :-1]
train_labels = train_data_full[:, -1]
test_data = test_data_full[:, :-1]
test_labels = test_data_full[:, -1]

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

In [ ]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [ ]:
train_data=train_data.astype("float32")/255
test_data = test_data.astype("float32")/255

In [ ]:
def build_model():
  model=models.Sequential()
  model.add(layers.Dense(100,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), input_shape=(784,)))
  model.add(layers.Dense(80,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
  model.add(layers.Dropout(.2))
  model.add(layers.Dense(80,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
  model.add(layers.Dropout(.2))
  model.add(layers.Dense(50,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
  model.add(layers.Dropout(.2))
  model.add(layers.Dense(40,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
  model.add(layers.Dense(5, activation="softmax"))
  model.compile(optimizer="rmsprop", loss = "categorical_crossentropy", metrics=["accuracy"])

  return model

In [ ]:
k= 4
num_val_sample = len(train_data) // k
num_epochs =100
all_scores = []
all_val_loss_histories = []
all_val_acc_histories = []
all_loss_histories =[]
all_acc_histories = []

for i in range(k):
  print("processing fold #",i)
  val_data = train_data[i*num_val_sample:(i+1)*num_val_sample]
  val_labels = train_labels[i*num_val_sample: (i+1)*num_val_sample]

  partial_train_data=np.concatenate([train_data[:i*num_val_sample], train_data[(i+1)*num_val_sample:]], axis=0)
  partial_train_labels=np.concatenate([train_labels[:i*num_val_sample], train_labels[(i+1)*num_val_sample:]], axis=0)

  model=build_model()
  history = model.fit(partial_train_data, partial_train_labels, validation_data = (val_data, val_labels), epochs= num_epochs, batch_size=512, verbose=1)
  val_loss, val_acc = model.evaluate(test_data, test_labels, verbose=0)
  val_loss_history = history.history["val_loss"]
  val_acc_history = history.history["val_accuracy"]
  loss_history = history.history["loss"]
  acc_history = history.history["accuracy"]
  all_loss_histories.append(loss_history)
  all_acc_histories.append(acc_history)
  all_val_loss_histories.append(val_loss_history)
  all_val_acc_histories.append(val_acc_history)
  all_scores.append(val_loss)
ave_val_loss_hist = [np.mean([x[i] for x in all_val_loss_histories]) for i in range(num_epochs)]
ave_loss_hist = [np.mean([x[i] for x in all_loss_histories]) for i in range(num_epochs)]
ave_val_acc_hist = [np.mean([x[i] for x in all_val_acc_histories]) for i in range(num_epochs)]
ave_acc_hist = [np.mean([x[i] for x in all_acc_histories]) for i in range(num_epochs)]
all_scores

In [ ]:
plt.plot(range(1, len(ave_val_loss_hist)+1)[20:], ave_val_loss_hist[20:], "bo", label="Validation Loss")
plt.plot(range(1, len(ave_val_loss_hist)+1)[20:], ave_loss_hist[20:], "b", label="Training Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.plot(range(1, len(ave_val_acc_hist)+1)[20:], ave_val_acc_hist[20:], "bo", label="Validation Accuracy")
plt.plot(range(1, len(ave_val_acc_hist)+1)[20:], ave_acc_hist[20:], "b", label="Training Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


In [ ]:
model=models.Sequential()
model.add(layers.Dense(100,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), input_shape=(784,)))
model.add(layers.Dense(80,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
model.add(layers.Dropout(.2))
model.add(layers.Dense(80,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
model.add(layers.Dropout(.2))
model.add(layers.Dense(50,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
model.add(layers.Dropout(.2))
model.add(layers.Dense(40,kernel_regularizer=regularizers.l1_l2(l1=.001, l2=.001), activation = "relu" ))
model.add(layers.Dense(5, activation="softmax"))
model.compile(optimizer="rmsprop", loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(train_data,train_labels, epochs =100, batch_size=512, verbose=1)

In [ ]:
test_loss_score, test_acc_score=model.evaluate(test_data, test_labels)

In [ ]:
prediction=model.predict(test_data)

In [ ]:
prediction_result = []
for i, v in enumerate(prediction):
  prediction_result.append(np.argmax(prediction[i]))
print(prediction_result[15:25])

test_labels_result = []
for i, v in enumerate(test_labels):
  test_labels_result.append(np.argmax(test_labels[i]))
print(test_labels_result[15:25])

In [ ]:
matched = []
for i, v in enumerate(test_labels_result):
  if  prediction_result[i] == test_labels_result[i]:
    matched.append("Yes")
  else:
    matched.append("No")

print("There are",matched.count("Yes"), "matched flowers and there are", matched.count("No"), "unmatched flowers ")